In [ ]:
!git --version


git version 2.34.1


In [ ]:
# Step 1: Clone the repository with sparse checkout enabled
!git clone --depth=1 --filter=blob:none --sparse https://github.com/ed-donner/llm_engineering.git

# Step 2: Move into the repository
%cd llm_engineering

# Step 3: Enable sparse checkout & get only "week5/knowledge-base"
!git sparse-checkout set week5/knowledge-base
!git pull


fatal: destination path 'llm_engineering' already exists and is not an empty directory.
/content/llm_engineering
Already up to date.


In [ ]:
import os
import glob
# from dotenv import load_dotenv
# import gradio as gr
from openai import OpenAI

In [ ]:
import google.generativeai as genai

# Set your Gemini API key (assuming it's stored in a variable named `API_KEY`)
API_KEY = "your-api-key"  # Replace this with your actual key

# Configure the Gemini API
genai.configure(api_key=API_KEY)

# Select the model
MODEL = "gemini-1.5-flash"




In [ ]:
context = {}

employees = glob.glob("/content/llm_engineering/week5/knowledge-base/employees/*")
# /content/llm_engineering/week5/knowledge-base/employees
for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc





In [ ]:
products = glob.glob("/content/llm_engineering/week5/knowledge-base/products/*")

In [ ]:
for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [ ]:
context.keys()

dict_keys(['Greene', 'Carter', 'Lancaster', 'Bishop', 'Harper', 'Trenton', 'Spencer', 'Tran', 'Blake', 'Chen', 'Thomson', 'Thompson', 'Carllm', 'Markellm', 'Homellm', 'Rellm'])

In [ ]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context

# get_relevant_context("Who is lancaster?")
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message
# print(add_context("Who is Alex Lancaster?"))

In [ ]:
# !pip install -U langchain-community


In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
MODEL
db_name = "vector_db"

In [ ]:
import os
import glob
from langchain.document_loaders import DirectoryLoader, TextLoader

folders = glob.glob("/content/llm_engineering/week5/knowledge-base/*/")

text_loader_kwargs = {'encoding': 'utf-8'}
documents = []

for folder in folders:
    doc_type = os.path.basename(os.path.normpath(folder))  # Fix applied
    print(f"Folder: {folder}, Extracted doc_type: {doc_type}")  # Debugging step

    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()

    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type  # Assign doc_type correctly
        documents.append(doc)

# Check if doc_type is correctly assigned
for doc in documents[:5]:  # Print first 5 docs
    print(doc.metadata)


Folder: /content/llm_engineering/week5/knowledge-base/employees/, Extracted doc_type: employees
Folder: /content/llm_engineering/week5/knowledge-base/company/, Extracted doc_type: company
Folder: /content/llm_engineering/week5/knowledge-base/contracts/, Extracted doc_type: contracts
Folder: /content/llm_engineering/week5/knowledge-base/products/, Extracted doc_type: products
{'source': '/content/llm_engineering/week5/knowledge-base/employees/Samantha Greene.md', 'doc_type': 'employees'}
{'source': '/content/llm_engineering/week5/knowledge-base/employees/Emily Carter.md', 'doc_type': 'employees'}
{'source': '/content/llm_engineering/week5/knowledge-base/employees/Avery Lancaster.md', 'doc_type': 'employees'}
{'source': '/content/llm_engineering/week5/knowledge-base/employees/Jordan K. Bishop.md', 'doc_type': 'employees'}
{'source': '/content/llm_engineering/week5/knowledge-base/employees/Alex Harper.md', 'doc_type': 'employees'}


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)


In [ ]:
doc_types = set(chunk.metadata.get('doc_type', 'Unknown') for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")


Document types found: products, company, employees, contracts


In [ ]:
# !pip install langchain openai
# !pip install langchain-chroma


In [ ]:
# !pip install langchain openai langchain-openai


In [ ]:

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
import langchain_openai

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [ ]:
# !pip uninstall -y numpy langchain transformers sentence-transformers langchain-huggingface


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: langchain 0.3.20
Uninstalling langchain-0.3.20:
  Successfully uninstalled langchain-0.3.20
Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
Found existing installation: sentence-transformers 3.4.1
Uninstalling sentence-transformers-3.4.1:
  Successfully uninstalled sentence-transformers-3.4.1


In [ ]:
# !pip install --no-cache-dir numpy==1.26.4
# !pip install --no-cache-dir langchain langchain-huggingface transformers sentence-transformers


In [ ]:
# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [ ]:
# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 130 documents


In [ ]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [ ]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [ ]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFaceHub

In [ ]:

llm = HuggingFaceHub(repo_id="EleutherAI/gpt-neo-2.7B", huggingfacehub_api_token="hf-token")

# Set up the conversation memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# The retriever is an abstraction over the VectorStore
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# Create the conversational retrieval chain
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "whos averi"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.



Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

- **2020**: *Meets Expectations*  
  Maxine focused on regaining her footing and excelling with technical skills. She was stable, though not standout, in her contributions. Feedback indicated a need for more proactivity.  

- **2021**: *Exceeds Expectations*  
  Maxine spearheaded the transition to a new data warehousing solution, significantly enhancing Insurellm’s data analytics capabilities. This major achievement bolstered her reputation within the company.  

- **2022**: *Outstanding*  
  Maxine continued her upward trajectory, successfully implementing machine learning algorithms to predict customer behavior, which was well-received by the leadership team and improved client satisfaction.  

- **2023**: *Exceeds Expectations*  
  Maxine has taken on mentoring responsibilities and is leading a cross-functional team for 

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
import gradio as gr

In [ ]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True,debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f2e83dc0d08dc0b918.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_depre